In [2]:
import pandas as pd
import re
import spacy
from nltk.corpus import stopwords
import nltk

In [3]:
df = pd.read_csv("/home/afpuertav/mySpace/archivos/tweetsCompletos20250616.csv")

In [4]:
df.drop(
    columns=["Sentimiento", "Latitude", "Longitude", "Emocion", "Content_limpio",
             "nmf_topic_id", "nmf_topic_keywords", "cantidad de palabras", "perplejidad"],
    errors="ignore",
    inplace=True
)


In [ ]:
df

,Author,Content,Date,Language,Location,Number of Likes,Number of Retweets,In Reply To,Author Name,Author Description,Author Statuses Count,Author Favourites Count,Author Friends Count,Author Followers Count,Author Listed Count,Author Verified
0,@quepasaraucania,[COVID-19 EN LA ARAUCANÍA 🦠] La Araucanía aman...,2020-06-03 18:56:49,es,NaN,0.0,0.0,NaN,Qué pasa Araucanía,"Medio independiente, sin tendencia política ni...",417.0,9.0,129.0,235.0,0.0,False
1,@Digital_58,Alarma en el deporte japones tras confirmarse ...,2020-06-03 18:56:47,es,NaN,0.0,0.0,NaN,Noticias Digital58 🇻🇪,#PeriodismoWebDeVerdad\n✉ prensa@digital58.com...,43440.0,15905.0,2902.0,6264.0,66.0,False
2,@SIPSENOTICIAS,"#SNCUN Hasta las 12 horas del 03 de junio, se ...",2020-06-03 18:56:45,es,NaN,0.0,0.0,NaN,SIPSE NOTICIAS,Grupo SIPSE empresa líder en información en el...,141180.0,1998.0,925.0,39818.0,281.0,False
3,@Olivareslucia,"#LaPreguntaExacta ¿Qué temas, además de la pan...",2020-06-03 18:56:44,es,NaN,0.0,0.0,NaN,Lucía Olivares,Comunicadora. Autora de #SeñoraBruja. Me escuc...,10679.0,1808.0,2261.0,1990.0,9.0,False
4,@seleneverar_,* Tania Tinoco a Xavier Vallarino: ¿puede deci...,2020-06-03 18:56:41,es,EC,0.0,0.0,NaN,Selene Vera R.,#Periodista y #Dircom •Sin fines de semana •En...,5696.0,5875.0,1871.0,316.0,8.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219016,@MunicipioEE,Informe N° 265. 15 detenidos en controles en v...,2020-05-09 13:15:32,es,NaN,1.0,0.0,NaN,Esteban Echeverría,Cuenta oficial del Municipio de Esteban Echeve...,9400.0,865.0,186.0,9682.0,38.0,False
1219017,@bbva,¿Puede transmitirse la #COVID19 a través del a...,2020-05-09 13:15:30,es,NaN,10.0,2.0,NaN,BBVA,"Cuestionamos lo establecido, buscamos nuevos c...",103676.0,1717.0,294.0,2553881.0,5711.0,True
1219018,@alexlibre18,"@HLGatell Ok Hugo, te lo compro. \n\nAhora bie...",2020-05-09 13:15:25,es,NaN,0.0,0.0,@HLGatell,LEX,libertad de expresión,1772.0,775.0,598.0,136.0,0.0,False
1219019,@30_marval,Evita reuniones y fiestas durante la emergenci...,2020-05-09 13:15:21,es,NaN,0.0,0.0,NaN,El profe marval,"NACI EN GDL,REGISTRADO EN LA DELEGACION DE SAN...",720.0,7.0,296.0,81.0,0.0,False


In [ ]:
nltk.download("stopwords")
stop_words = set(stopwords.words("spanish"))

nlp = spacy.load("es_core_news_sm")

def limpiar_texto(texto):
    if pd.isna(texto):
        return ""
    
    # Pasar a minúsculas
    texto = texto.lower()
    
    # Eliminar URLs
    texto = re.sub(r"http\S+|www\S+|https\S+", "", texto)
    
    # Eliminar menciones y hashtags
    texto = re.sub(r"@\w+|#\w+", "", texto)
    
    # Eliminar caracteres no alfabéticos (mantener tildes y ñ)
    texto = re.sub(r"[^a-záéíóúüñ\s]", " ", texto)
    
    # Eliminar espacios extra
    texto = re.sub(r"\s+", " ", texto).strip()
    
    # Tokenizar, lematizar y quitar stopwords
    doc = nlp(texto)
    tokens = [token.lemma_ for token in doc if token.lemma_ not in stop_words and len(token.lemma_) > 2]
    
    return " ".join(tokens)
# Crear nueva columna limpia
df["Content_cleaned"] = df["Content"].apply(limpiar_texto)


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/afpuertav/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
gazetteer = [
    # Departamentos
    "amazonas", "antioquia", "arauca", "atlántico", "bolívar", "boyaca", "caldas", "caqueta",
    "casanare", "cauca", "cesar", "choco", "cordoba", "cundinamarca", "guainía", "guaviare",
    "huila", "la guajira", "magdalena", "meta", "nariño", "norte de santander", "putumayo",
    "quindio", "risaralda", "san andres", "santander", "sucre", "tolima", "valle del cauca",
    "vaupes", "vichada",

    # Capitales de departamento
    "leticia", "medellin", "arauca", "barranquilla", "cartagena", "tunja", "manizales",
    "florencia", "yopal", "popayan", "valledupar", "quibdo", "monteria", "bogota",
    "inirida", "san jose del guaviare", "neiva", "riohacha", "santa marta", "villavicencio",
    "pasto", "cucuta", "mocoa", "armenia", "pereira", "san andres", "bucaramanga",
    "sincelejo", "ibague", "cali", "mitu", "puerto carreno", "puerto carreño",

    # Ciudades adicionales relevantes
    "soledad", "bello", "envigado", "palmira", "itagüí", "cartago", "tulúa",
    "maicao", "sabanalarga", "malambo", "puerto colombia", "soledad", "bello",
    

    # Pueblos patrimonio y con historia
    "santa cruz de mompox", "cienaga", "aguadas", "salamina", "barichara", "jardin",
    "villa de leyva", "guadalajara de buga", "jerico", "giron", "mongui", "honda",
    "playa de belen", "pore", "guaduas", "lorica", "santa fe de antioquia", "el socorro",

    # Parques nacionales y reservas naturales
    "tayrona", "chiribiquete", "nevados", "cocuy", "purace", "serrania de la macarena",
    "caño cristales", "amacayacu", "gorgona", "utria", "tatacoa", "corales del rosario",
    "guacharos", "tuparro", "isla de salamanca", "paramillo", "katios", "otun quimbaya",
    "malpelo", "santuario de fauna y flora galeras", "isla de la corota", "corota", 
    "nukak", "yariguies"
]


In [ ]:
jerga = [
    "cascar", "farra", "farrear", "farreando", "guaro", "guayabo", "jincho", "trago", "pola", "whiskeria",
    "balde", "berraquera", "berraco", "caneca", "cachucha", "cantaleta", "cantaletoso", "cantaletosa",
    "camello", "camellar", "camellando", "chaguala", "chepa", "chechere", "chiripa", "chimba", "chimbo",
    "chucha", "chuspa", "cucos", "cumbamba", "despelote", "drogueria", "entuque", "filo", "guevonada", "aguevonado",
    "jartera", "jarto", "jarta", "locha", "melona", "motoso", "mota", "monda", "cule", "paila", "pecueca", "tombo", 
    "tombos", "aguevonada", "aguevonadas", "aguevonados", "trancon", "trola", "vaina", "vainas", "verguero", "achantao", "achantada", "atolondrado", "atolondrada",
    "bacano", "boleta", "canson", "chino", "cucho", "culicagao", "chanda", "chichipato", "ficti", "gamin", "gomelo",
    "guevon", "guevona", "ingrimo", "lacra", "lampara", "llave", "mono", "mamon",
    "metido", "nero", "parcero", "pelao", "picho", "pirobo", "piroba", "pulido",
    "toche", "sapo", "vieja", "abrirse", "amanarse", "dar papaya", "camellar", 
    "correrse", "destutanarse", "encularse", "embolatarse", "estar pilas", "hacer una vaca", 
    "pedir un catorce", "manejar", "ponerse las pilas", "parar bolas", "rumbear",
    "tirar caja", "tirar los perros", "vitriniar", "atembado", "bergajo", "carranguero", 
    "guache", "guambito", "mecato", "paila", "transmilenio"
]


NameError: name 'df' is not defined